In [1]:
from colour import Color
from chalk.core import BaseDiagram
from chalk import *

def help(f):
    import pydoc
    from IPython.display import HTML
    return HTML(pydoc.HTMLDoc().docroutine(f))

set_svg_height(100)

Chalk supports basic methods for complex connected layouts and diagrams.
Individual elements can be assigned names, and then be referenced in their subdiagram locations.
As we will see, names are particularly useful for connecting different parts of the diagram with arrows.
Our implementation follows the API of the Haskell [diagrams](https://diagrams.github.io/doc/manual.html#named-subdiagrams) library,
but named nodes are also common in TikZ.

### Diagram.named

In [2]:
help(BaseDiagram.named)

In [3]:
(square(1).show_origin() + chalk.arrowheads.dart()).render_svg("tri.svg", 56)

[[18.56913841  0.        ]
 [ 0.         18.56913841]] 344.8129013380807


In [4]:
diagram = circle(0.5).named("x") | square(1)
diagram

### Diagram.get_subdiagram

In [5]:
help(BaseDiagram.get_subdiagram)

A `Subdiagram` is a `Diagram` paired with its enclosing context (a `Transformation` for the moment; but `Style` should also be added at some point).
It has the following methods:
- `get_envelope`, which returns the corresponding `Envelope`
- `get_trace`, which returns the corresponding `Trace`
- `get_location`, which returns the local origin of the `Subdiagram`
- `boundary_from`, which return the furthest point on the boundary of the `Subdiagram`, starting from the local origin of the `Subdigram` and going in the direction of a given vector.

In [6]:
diagram = circle(0.5).named("x") | square(1)
sub = diagram.get_subdiagram("x")
diagram + circle(0.2).translate_by(sub.get_location())

### Diagram.with_names

In [7]:
help(BaseDiagram.with_names)

In [8]:
root = circle(1).named("root")
leaves = hcat([circle(1).named(c) for c in "abcde"], sep=0.5).center()

In [9]:
def connect(subs, nodes):
    root, leaf = subs
    pp = root.boundary_from(unit_y)
    pc = leaf.boundary_from(-unit_y)
    return nodes + Path.from_points([pp, pc]).stroke()

In [10]:
nodes = root / vstrut(2) / leaves

In [11]:
for c in "abcde":
    nodes = nodes.with_names(["root", c], connect)
nodes

### Diagram.qualify

In [12]:
help(BaseDiagram.qualify)

In [13]:
red = Color("red")

In [14]:
def attach(subs, dia):
    sub1, sub2 = subs
    p1 = sub1.get_location()
    p2 = sub2.get_location()
    return dia + Path.from_points([p1, p2]).stroke().line_color(red)

In [15]:
def squares():
    s = square(1)
    return (
        (s.named("NW") | s.named("NE")) /
        (s.named("SW") | s.named("SE")))

In [16]:
dia = hcat([squares().qualify(str(i)) for i in range(5)], sep=0.5)
pairs = [
    (("0", "NE"), ("2", "SW")),
    (("1", "SE"), ("4", "NE")),
    (("3", "NW"), ("3", "SE")),
    (("0", "SE"), ("1", "NW")),
]

In [17]:
dia

In [18]:
for pair in pairs:
    print(pair)
    dia = dia.with_names(pair, attach)

(('0', 'NE'), ('2', 'SW'))
(('1', 'SE'), ('4', 'NE'))
(('3', 'NW'), ('3', 'SE'))
(('0', 'SE'), ('1', 'NW'))


In [19]:
dia

### Diagram.show_labels

In [20]:
help(BaseDiagram.show_labels)

In [21]:
dia.show_labels(font_size=0.2)

### Diagram.connect

In [22]:
help(BaseDiagram.connect)

In [23]:
diagram = circle(0.5).named("x") | hstrut(1) | square(1).named("y")
diagram.connect("y", "x")

[[-47.52390486   0.09490555]
 [ -0.09490555 -47.52390486]] 2258.5305398644946
[[-47.52390486   0.09490555]
 [ -0.09490555 -47.52390486]] 2258.5305398644946


### Diagram.connect_outside

In [24]:
help(BaseDiagram.connect_outside)

In [25]:
diagram = circle(0.5).named("x") | hstrut(1) | square(1).named("y")
d = diagram.connect_outside("x", "y")
d.render_svg("connected.svg")
d

[[60.83023481 -0.24283624]
 [ 0.24283624 60.83023481]] 3700.3764365139896
[[47.52362095 -0.18971581]
 [ 0.18971581 47.52362095]] 2258.5305398644928
[[47.52362095 -0.18971581]
 [ 0.18971581 47.52362095]] 2258.5305398644928
